In [1]:
import pandas as pd

df = pd.read_csv('data/imputed_data.csv').set_index('id')
df

loan_amnt  funded_amnt_inv  int_rate  installment  grade  sub_grade  \
id                                                                              
68407277     3600.0           3600.0     13.99       123.03   20.0       24.0   
68355089    24700.0          24700.0     11.99       820.28   20.0       21.0   
68476807    10400.0          10400.0     22.45       289.91   50.0       51.0   
68426831    11950.0          11950.0     13.44       405.18   20.0       23.0   
68476668    20000.0          20000.0      9.17       637.58   10.0       12.0   
...             ...              ...       ...          ...    ...        ...   
89905081    18000.0          18000.0      9.49       377.95   10.0       12.0   
88948836    29400.0          29400.0     13.99       683.94   20.0       23.0   
89996426    32000.0          32000.0     14.49       752.74   20.0       24.0   
90006534    16000.0          16000.0     12.79       362.34   20.0       21.0   
88224441    24000.0          24000.0     14.49       564.56   20.0       24.0   

          emp_length  home_ownership  annual_inc  verification_status  ...  \
id                                                                     ...   
68407277        10.0             3.0     55000.0                  0.0  ...   
68355089        10.0             3.0     65000.0                  0.0  ...   
68476807         3.0             3.0    104433.0                  2.0  ...   
68426831         4.0             4.0     34000.0                  2.0  ...   
68476668        10.0             3.0    180000.0                  0.0  ...   
...              ...             ...         ...                  ...  ...   
89905081         5.0             5.0    130000.0                  0.0  ...   
88948836         9.0             3.0    180792.0                  0.0  ...   
89996426         3.0             3.0    157000.0                  2.0  ...   
90006534        10.0             4.0    150000.0                  0.0  ...   
88224441         6.0             4.0    110000.0                  0.0  ...   

          num_tl_op_past_12m  pct_tl_nvr_dlq  percent_bc_gt_75  \
id                                                               
68407277                 3.0            76.9               0.0   
68355089                 2.0            97.4               7.7   
68476807                 4.0            96.6              60.0   
68426831                 0.0           100.0             100.0   
68476668                 2.0            96.3             100.0   
...                      ...             ...               ...   
89905081                 0.0           100.0              33.3   
88948836                 2.0           100.0              42.9   
89996426                 2.0           100.0               0.0   
90006534                 2.0            92.0              50.0   
88224441                 1.0            96.2              40.0   

          pub_rec_bankruptcies  tax_liens  tot_hi_cred_lim  total_bal_ex_mort  \
id                                                                              
68407277                   0.0        0.0         178050.0             7746.0   
68355089                   0.0        0.0         314017.0            39475.0   
68476807                   0.0        0.0         439570.0            95768.0   
68426831                   0.0        0.0          16900.0            12798.0   
68476668                   0.0        0.0         388852.0           116762.0   
...                        ...        ...              ...                ...   
89905081                   0.0        0.0         275356.0            54349.0   
88948836                   0.0        0.0         719056.0           148305.0   
89996426                   0.0        0.0         524379.0           122872.0   
90006534                   3.0        0.0          87473.0            65797.0   
88224441                   1.0        0.0          84664.0            62426.0   

          total_bc_limit  to

## Outlier analysis using SKLearn.

In [2]:
#Lets try to use sklearn to detect outliers

from sklearn.ensemble import IsolationForest

outlier_percentage = 0.02

# Fit Isolation Forest to detect outliers
iso_forest = IsolationForest(contamination=outlier_percentage, random_state=42)
o1 = iso_forest.fit_predict(df)
o1

array([1, 1, 1, ..., 1, 1, 1])

In [3]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(contamination=outlier_percentage)
o2 = lof.fit_predict(df)

o2


array([1, 1, 1, ..., 1, 1, 1])

In [4]:
#adding both series
import numpy as np

print(f'Outliers detected by each algorithms: {np.count_nonzero(o1 == -1)}')
outliers = o1 + o2
print(f'Outliers detected by both algorithms: {np.count_nonzero(outliers == -2)}')

df1 = df[outliers != -2] # == -2 because it means this has been identified as outlier by both the algorithms and removing only those

Outliers detected by each algorithms: 24736
Outliers detected by both algorithms: 1394


In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# Display the original and updated DataFrames
print(f'Before outlier detection : {df.shape[0]} rows')
print(f'After outlier detection : {df1.shape[0]} rows')

Before outlier detection : 1236793 rows
After outlier detection : 1235399 rows


In [6]:
df1['loan_status'].value_counts()

1.0    987018
0.0    248381
Name: loan_status, dtype: int64

In [7]:
df1.to_csv('data/removed_outlier.csv')